In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.special as special
import time
import matplotlib.pyplot as plt

params = {'font.size': 15,
          'axes.linewidth':2}
plt.rcParams.update(params)

# Implementing the Tail Sensitive test in python

## Defining function to calculate the test statistic T
First to test against the standard normal distribution

In [ ]:
# function make_ts_c
# Calculates the distriubtion of the Tail Sensitive test statistic
# Input:  m: Number of samples from the statistic distribution
#         n: Size of the sample that will that will be tested
# Output: c: Sorted array containing an m size sample of the Tail Sensitive
#            test statistic T distribution.
def make_ts_c(m,n):
    uniforms =  np.sort(stats.uniform.rvs(size=(m,n)))
    indices = np.arange(n)+1
    b = special.betainc(indices,n+1-indices,uniforms)
    ci = np.minimum(b,1-b)
    c = np.sort(2*np.amin(ci,axis=1))
    return c

In [ ]:
# function make_ks_c
# Calculates the distriubtion of the Kolgomorov Smirnov test statistic
# Input:  m: Number of samples from the statistic distribution
#         n: Size of the sample that will that will be tested
# Output: c: Sorted array containing an m size sample of the Kolmogorov-Smirnov
#            test statistic T distribution.
def make_ks_c(m,n):
    uniforms =  np.sort(stats.uniform.rvs(size=(m,n)))
    indices = np.arange(n)+1
    t_quantiles = indices/(n+1)
    t_ks = np.absolute(t_quantiles-uniforms)
    c = np.sort(np.amax(t_ks,axis=1))
    return c

## Defining function to create a qq plot and perform the tests

In [ ]:
# function qq_plot
# Creates a quantile quantile plot of a given sample comparing to a certain distribution and performs 
# the Kolmogorov-Smirnov and Tail Sensitive tests
# Input:  x - numpy array containig a sample from a certain distribution.
#         alpha - real number. Confidence level of the hypothesis test
#         m - integer. Number of simulations to calculate the p-value 
#                      (The larger m the more accurate the p-value but also a larger computation time)
#         dist - scipy.stats distribution. Distribution to which compare the given sample
#         rotated - Logical. If false the result will be a regular qq_plot 
#                   mapping theoretical and empirical quantiles. If True
#                   the plot will be rotated 45 degrees clockwise
#                   rotated = True is recommended for large sample size, this makes the discrepancies between
#                   theoretical and observed quantiles easier to see.
#         TS - Logical. If true will perform the Tail-sensitive test, Add the corresponding 
#              confidence bands to the qq plot and print the p-value, test statistic and
#              critical value
#         KS - Logical. If true will perform the Kolgomorov-Smirnov test, Add the corresponding 
#              confidence bands to the qq plot and print the p-value, test statistic and
#              critical value
#         smaple_label - string. Label for the legend representing the sample
#         filename - string. Name for the file where the figure is saved (format is pdf)
def qq_plot(x, alpha = 0.05, m = 5000, dist = stats.norm, rotated = True, TS = True, KS = True,
            sample_label = "Sample", filename='qqplot.pdf'):
    n = len(x)
    indices = np.arange(n) + 1
    u_quantiles = indices/(n+1)
    n_quantiles = dist.ppf(u_quantiles)
    sortedx = np.sort(x)
    uniforms = dist.cdf(sortedx)

    if TS:
        c = make_ts_c(m,n)
        gamma = c[int(alpha*m)]
        b = special.betainc(indices,n+1-indices,uniforms)
        lower = special.betaincinv(indices,n+1-indices,0.5*gamma)
        upper = special.betaincinv(indices,n+1-indices,1-0.5*gamma)
        ci = np.minimum(b,1-b)
        t = 2*ci.min()
        p_value = np.searchsorted(c,t)/m
        n_lower = dist.ppf(lower)
        n_upper = dist.ppf(upper)
        
    if KS:
        c_ks = make_ks_c(m,n)
        gamma_ks = c_ks[int((1-alpha)*m)]
        t_ks = np.absolute(u_quantiles-uniforms).max()
        lower_ks = u_quantiles - gamma_ks
        upper_ks = u_quantiles + gamma_ks
        n_lower_ks = dist.ppf(lower_ks)
        n_upper_ks = dist.ppf(upper_ks)
        p_value_ks = 1-np.searchsorted(c_ks,t_ks)/m
        
    fig, ax = plt.subplots()
    ir = 0
    if rotated:
        ir = 1
    ax.plot(n_quantiles,sortedx-n_quantiles*ir,'o',label=sample_label)
    ax.set_xlabel(r'$Q_{\rm theo}$')
    
    if rotated:
        ax.axhline(y=0, color='k')
        ax.set_ylabel(r'$Q_{\rm exp} - Q_{\rm theo}$')
    else:
        x0,x1 = ax.get_xlim()
        y0,y1 = ax.get_ylim()
        x0 = min(x0,y0)
        x1 = max(x1,y1)
        ax.set_xlim(x0,x1)
        ax.set_ylim(x0,x1)
        ax.set_aspect('equal')
        diagonal = np.linspace(x0,x1,100)
        ax.plot(diagonal,diagonal,color='k')
        ax.set_ylabel(r'$Q_{\rm exp}$')
    
    if TS:
        ax.plot(n_quantiles,n_lower-n_quantiles*ir,color='C1',label="TS test")
        ax.plot(n_quantiles,n_upper-n_quantiles*ir,color='C1')
        
    if KS:
        ax.plot(n_quantiles,n_lower_ks-n_quantiles*ir,color='C2',label="KS test")
        ax.plot(n_quantiles,n_upper_ks-n_quantiles*ir,color='C2')
    

    
    ax.legend()
    
    plt.savefig(filename,format='pdf',bbox_inches='tight',transparent=True)
    
    plt.show()
    if TS:
        print("Tail Sessitive test. alpha = ", alpha)
        print("p-value        = ", p_value)
        print("Test statistic = ", t)
        print("Critical value = ", gamma)
    if KS:
        if TS:
            print()
        print("Kolgomorov Smirnov test. alpha = ", alpha)
        print("p-value        = ", p_value_ks)
        print("Test statistic = ", t_ks)
        print("Critical value = ", gamma_ks)


### Example. Testing a sample against the normal distribution

In [ ]:
# Reading sample
sample = np.loadtxt('normal_sample_example.dat', usecols=[0])

# Making a histogram
y = np.linspace(-4,4,100)
n, bins, patches = plt.hist(sample,density=True)
l = plt.plot(y, stats.norm.pdf(y))

The histogram shows a "bell shape" that mostly agrees with the normal distribution. But a closer look with a qq-plot and a couple of normality tests can show a different picture

In [ ]:
qq_plot(sample, sample_label='Residuals')

#### Analysis of example
In this particular example the Kolmogorov Smirnov test does not find any statistically significant deviations between the sample and the normal distribution. (p-value > alpha)

In contrast, the Tail-Sensitive test does see a statistically significant deviation between the sample and a normal distribution.

### Example 2. Testing against a $\chi^2$ distribution

In this particular example the $\chi^2$ has 49 degrees of freedom

In [ ]:
sample_2 = np.loadtxt('chi2_sample_example.dat')
y = np.linspace(20,130, 100)
df = 49
n, bins, patches = plt.hist(sample_2,density=True)
l = plt.plot(y, stats.chi2.pdf(y,df))
print('Mean = ',np.mean(sample_2))
print('Var  = ',np.var(sample_2))

In [ ]:
qq_plot(sample_2 ,dist=stats.chi2(df), sample_label=r'$\chi^2$ sample')

Notice how the shape of the KS and TS limits has changed now that the sample is being tested against a different distribution. 

Here both the Kolmogorov-Smirnov and Tail Sensitive test reject the hypothesis that the sample comes from a $\chi^2$ distribution with 49 degrees of freedom